# Spotify Web API

In [1]:
import json
from yaml import safe_load
from typing import Final
from spotipy import Spotify
from spotipy.oauth2 import SpotifyOAuth
from functools import lru_cache

## Read client data from conf.yml file

In [2]:
def read_client_data() -> dict[str, str]:
    with open('conf.yml', 'r') as f:
        data = safe_load(f)
        return data
    
client_data: dict[str, str] = read_client_data()
CLIENT_ID: Final[str] = client_data.get('client_id')
CLIENT_SECRET: Final[str] = client_data.get('client_secret')
REDIRECT_URI: Final[str] = client_data.get('redirect_uri')

## Create Spotify object with client authorization

In [3]:
# Authenticate with Spotify using OAuth
scopes="user-read-email,user-library-read,user-top-read,user-read-recently-played"
auth_manager=SpotifyOAuth(client_id=CLIENT_ID, client_secret=CLIENT_SECRET, redirect_uri=REDIRECT_URI, scope=scopes)
sp = Spotify(auth_manager=auth_manager)

## Get User

In [ ]:
sp.me()

## Top Tracks

### Get current user top tracks in the short term, medium term and long term.

In [4]:
# Short term
top_tracks = sp.current_user_top_tracks(time_range='short_term', limit=50)

In [ ]:
# Medium term
top_tracks = sp.current_user_top_tracks(time_range='medium_term', limit=50)

In [ ]:
# Long term
top_tracks = sp.current_user_top_tracks(time_range='long_term', limit=50)

### Get list of top tracks and the artists

In [ ]:
list(map(lambda x: f"{x['artists'][0]['name']} - {x['name']}", top_tracks['items']))

### analyze the top tracks and return a dict of each artist and his tracks

In [9]:
# artists <-> tracks analyze with artist is the most accured in user top tracks
from copy import deepcopy
def artists_tracks_analyze(top_tracks: dict) -> dict[str, str]:
    tracks = deepcopy(top_tracks)
    artists_tracks = {}
    for i, item in enumerate(tracks['items']):
        print(f"{i}. {item['name']} - {item['artists'][0]['name']}")
        artist_name = item['artists'][0]['name']
        track_name = item['name']
        if artist_name not in artists_tracks.keys():
            artists_tracks[artist_name] = [track_name]
        else:
            artists_tracks[artist_name].append(track_name)

    return artists_tracks
     

In [ ]:
artists_tracks = artists_tracks_analyze(top_tracks)

In [ ]:
artists_tracks

### Count for each artist how muck songs he have in user top tracks in given term

In [ ]:
for artist, tracks in artists_tracks.items():
    print(f"{artist} , {len(tracks)}")

## Saved Tracks

### Get all saved tracks of current user

In [4]:
# Wrapper function for saved tracks to reduce api calls
@lru_cache(maxsize=1)
def get_saved_tracks():
    saved_tracks = []
    tracks = sp.current_user_saved_tracks()
    while tracks:
        for i, item in enumerate(tracks['items']):
           saved_tracks.append(item)
           track = item['track']
           if i % 10 == 0:
               print(i, track['artists'][0]['name'], ' - ', track['name'], end="\n")
           else:
              print(i, track['artists'][0]['name'], ' - ', track['name'], end=" ")
        
        if tracks['next']:
            tracks = sp.next(tracks)
        else:
            tracks = None
    return saved_tracks


In [ ]:
saved_tracks = get_saved_tracks()

In [ ]:
for track in saved_tracks:
    print(track['track']['name'], end="\n")

In [49]:
top_tracks = sp.current_user_top_tracks(time_range='short_term')

In [ ]:
for item in top_tracks['items']:
    print(json.dumps(item))

## Recently Played Tracks

In [12]:
# Wrapper function to get recetly played tracks
@lru_cache(maxsize=1)
def get_recently_played(limit: int = 50, before: str = None, after: str = None):
    recently_played = []
    tracks = sp.current_user_recently_played(limit=limit, after=after, before=before)
    while tracks:
        for i, item in enumerate(tracks['items']):
           recently_played.append(item)
           track = item['track']
           if i % 10 == 0:
               print(i, track['artists'][0]['name'], ' - ', track['name'], end="\n")
           else:
              print(i, track['artists'][0]['name'], ' - ', track['name'], end=" ")
        
        if tracks['next']:
            tracks = sp.next(tracks)
        else:
            tracks = None
    return recently_played

## Top Artists

### Get current user top 10 artists

In [11]:
ranges = ['short_term', 'medium_term', 'long_term']

for range in ranges:
    print(f'{range= }')

    top_artists = sp.current_user_top_artists(time_range=range, limit=10)

    for i, item in enumerate(top_artists['items']):
        print(f"{i}. {item['name']}")
    print()

range= 'short_term'
0. Eminem
1. NF
2. Twenty One Pilots
3. Imagine Dragons
4. Arctic Monkeys
5. Red Hot Chili Peppers
6. David Guetta
7. Coldplay
8. Green Day
9. Queen

range= 'medium_term'
0. Twenty One Pilots
1. Eminem
2. Queen
3. Red Hot Chili Peppers
4. Arctic Monkeys
5. Nirvana
6. Green Day
7. David Guetta
8. NF
9. Imagine Dragons

range= 'long_term'
0. Twenty One Pilots
1. Nirvana
2. Arctic Monkeys
3. Red Hot Chili Peppers
4. Eminem
5. Queen
6. R.E.M.
7. Green Day
8. The Beatles
9. The Weeknd

